## DATA
Analyze **two datasets** from the _Federal Student Aid office's Title IV Program Volume Reports_: 
* Loan Volume from the Direct Loan Program from the first quarter of academic years 2010-2011
* Loan Volume from the Direct Loan Program from the first quarter of academic years 2015-2016

Datasets are in CSV formats and are accessible via [ADA School's Dropbox](https://www.dropbox.com/sh/02pyg57qe9nbcqy/AACc_rMiZFlow4Tdxl1VntfKa?dl=0). Review the **READ ME** file before executing analysis and submitting your final application. 

## Cleaning the Data

**Exclude** all rows where:
* The school is designated as a foreign institution
* The final digit of the school's ZIP code is 3, 5, or 7

### _Question One_ 


> Consider all of the schools where

> `    $2,000 <=` **`loans disbursed`** `< $9,500`

> for the time period reported on the spreadsheets provided (i.e. quarter 1). How many more schools met this criteria in 2015 than in 2010?

Remember to provide your answer as a number. 

In [45]:
import pandas as pd

df10 = pd.read_csv('/Users/amanda/Desktop/Personal Work/DL_Dashboard_AY2010_2011_Q1.csv', header=[4,5])

L0 = df10.columns.get_level_values(0)
L1 = df10.columns.get_level_values(1)
combos = []
for i in range(len(L1)):
    if "Unnamed" in L0[i]:
        combos.append(L1[i])
    else:
       combos.append(L0[i] + L1[i])

df10.columns = combos
df10 = df10.dropna(subset=["OPE ID"])

df10 = df10.replace("\$", "", regex=True)
df10 = df10.replace("^\s+", "", regex=True)
df10 = df10.replace("\s+$", "", regex=True)
df10 = df10.replace("-", "0", regex=True)
df10["$ of Disbursements"] = df10["$ of Disbursements"].replace(",", "", regex=True)

df10["$ of Disbursements"] = df10["$ of Disbursements"].astype(float)

df10 = df10[~df10["School Type"].str.contains("FOREIGN", case=False)]

df10 = df10[~df10["Zip Code"].str[-1].isin(["3", "5", "7"])]
df10['Total $ Disbursements'] = df10[["$ of Disbursements"]].sum(axis=1)

In [46]:
num_10 = len(df10[(df10['Total $ Disbursements'] >= 2000) & (df10['Total $ Disbursements'] < 9500)])

In [47]:
df15 = pd.read_csv('/Users/amanda/Desktop/Personal Work/DL_Dashboard_AY2015_2016_Q1.csv', header=[4,5], encoding = "ISO-8859-1")
df15.head()
L0 = df15.columns.get_level_values(0)
L1 = df15.columns.get_level_values(1)
combos = []
for i in range(len(L1)):
    if "Unnamed" in L0[i]:
        combos.append(L1[i])
    else:
        combos.append(L0[i] + L1[i])

df15.columns = combos
df15 = df15.dropna(subset=["OPE ID"])

df15 = df15.replace("\$", "", regex=True)
df15 = df15.replace("^\s+", "", regex=True)
df15 = df15.replace("\s+$", "", regex=True)
df15 = df15.replace("-", "0", regex=True)
df15["$ of Disbursements"] = df15["$ of Disbursements"].replace(",", "", regex=True)

df15["$ of Disbursements"] = df15["$ of Disbursements"].astype(float)

df15 = df15[~df15["School Type"].str.contains("FOREIGN", case=False)]
df15["School Type"].unique()

df15 = df15[~df15["Zip Code"].str[-1].isin(["3", "5", "7"])]
df15['Total $ Disbursements'] = df15[["$ of Disbursements"]].sum(axis=1)

In [48]:
num_15 = len(df15[(df15['Total $ Disbursements'] >= 2000) & (df15['Total $ Disbursements'] < 9500)])

In [49]:
print(num_15 - num_10)

6


### 1.   Answer = 6
 * **process**: Using the pandas library within Python and Jupyter Notebooks, I solved this problem by reading in the CSVs for the Direct Loan Program from the first quarter of academic years 2010-2011 and 2015-2016. I removed the unnecessary header rows, and rows containing null values. I combined the unique loan types (e.g. undergraduate and graduate unsubsidized, subsidized, and parent plus) with recipient columns names by either ignoring the unnamed column values from the first Line of the header row (put another way, using the second line header row column name), or appending the loan type in the first line of the header to the recipient column name of the second line of the header row. This resulted in one header row of column names. Initial data cleaning required excluding (filtering the data) of foreign school types (by which I analyzed the unique School Types and filtered out any rows designated or containing FOREIGN) and institutions from zip codes where the final zip digit was 3,5,or 7. This questioned called for additional data cleaning, where in order to evaluate which  \$ of loan disbursements fell between 2000 and 9500 dollars, it was necessary to convert the cell's string to floats to perform a sum. Using regex, I filtered cell's special string characters (e.g. "$", "-", ",", " ") and replaced with a 0 value where appropriate. Once the cell input was converted to floats, then it was possible to sum all 6 \$ of loan disbursements across all rows to get a total \$ of disbursements. I kept track of this in a new column. This was performed on both CSV files (e.g. dataframe). Finally you could compare the number of schools who met this criteria to each other by counting the length of each dataframe and subtracting the difference of the Direct Loan Program 2015-2016 results from the Direct Loan Program 2010-2011.         
 * **assumptions**: The result of this question assumes that the desired outcome is a total \$ of disbursements that fell between 2000 and 9500 dollars per school, and not per recipient. It also assumes that all the data in each CSV is only related to quarter 1 disbursements. This result also assumes that I did not remove important data when removing null values.   
 * **choices**: To make computation easier, I used a regular expression (Regex), which is a special text string for describing a search pattern. Finding, removing, and replacing the special characters of that column allowed me to cast the strings to int and perform a sum operation over the column rows. Had I not done this, I would have had to manually add each row, or written an alternative program to handle this issue with the data. 
 * **reasoning**: I removed the unnecessary header rows and null values to avoid errors when processing the data, or having tens to hundreds of cells with unnecessary empty or 0 values. It makes analyzing the data much easier! I converted the strings to floats because you can't add (sum here) strings, making it very difficult to see which institutions' \$ of disbursements fell between 2000 and 9500 dollars, and the resulting difference in institutions meeting this criteria across the given Program academic years.


### _Question Two_

> Consider the sum of expected total loan amount if the loans were fully disbursed for each school. In 2010 as compared to 2015, for how many more schools was this amount greater than $20,000,000?

Remember to provide your answer as a number. 

In [50]:
df10 = df10.replace("\$", "", regex=True)
df10 = df10.replace("^\s+", "", regex=True)
df10 = df10.replace("\s+$", "", regex=True)
df10 = df10.replace("-", "0", regex=True)
df10["$ of Loans Originated"] = df10["$ of Loans Originated"].replace(",", "", regex=True)

df10["$ of Loans Originated"] = df10["$ of Loans Originated"].astype(float)
df10["Total $ of Loans Originated"] = df10[["$ of Loans Originated"]].sum(axis=1)
num_10 = len(df10[df10["Total $ of Loans Originated"] > 20000000])

In [51]:
df15 = df15.replace("\$", "", regex=True)
df15 = df15.replace("^\s+", "", regex=True)
df15 = df15.replace("\s+$", "", regex=True)
df15 = df15.replace("-", "0", regex=True)
df15["$ of Loans Originated"] = df15["$ of Loans Originated"].replace(",", "", regex=True)

df15["$ of Loans Originated"] = df15["$ of Loans Originated"].astype(float)
df15["Total $ of Loans Originated"] = df15[["$ of Loans Originated"]].sum(axis=1)
num_15 = len(df15[df15["Total $ of Loans Originated"] > 20000000])

In [53]:
print(num_10 - num_15)

1


### 2.   Answer = 1 
 * **process**: Like question one, in order to perform a sum across institutions \$ of Loans Originated, I used Regex to replace special string characters (e.g. "$", "-", ",", " ") in that column to perform a sum. I stored the sum in a  new coloum called "Total \$ of Loans Originated". Then I filtered the dataframe for the condition where the Total was greater than 20 million. This process was repeated for both CSV files. Finally, I subtracted the 2015-2016 total from the 2010-2011 total to find the given solution. 
 * **assumptions**: Unlike question one that was concerned with 2015-2016's output of dollars disbursed, the result of this question assumes that the desired result is looking for 2010-2011's number of institutions that exceeded 20 million when the expected total loan amount if the loan is fully disbursed. It is possible that I misinterpreted the question because the phrasing is difficult to interpret. Even though the remainder of questions are predominantly focused on the results of the 2015-2016 Direct Loan Program CSV, if we were to perform the comparison as such, the result would have resulted in a negative number.
 * **choices**: The question prompt eliminated many choices for analyzing this data. It calls for a sum of the expected total loan amount if the loans were fully disbursed for each school. This narrowed down the number of columns to evaluate across the dataframe to \$ of Loans Orginated. In order to treat each cell as a number, it required converted the data from a string to a number. It required filtering for a conditional statement (where the total must be larger than 20 million). Lastly, it called for a arithmetic operation to find the difference between years. If I wanted to add operations to solving this problem, I could have performed a count of the institutions that met this condition in each CSV after summing the totals across all appropriate columns, and then subtracted the counts from one another, but again, this would have been an arbitrary choice to subtract the result from on CSV from the other CSV.    
 * **reasoning**: To find the sum of expected total loan amounts if the loans were fully disbursed for each school, I reasoned the appropriate way to solve this problem was to take a sum across all columns of \$ of Loans Originated, store that total sum in a new column. Then repeat this down through each row (e.g. institution). Then filter the data for the condition of being greater than 20 million. Finally, after repeating this process across both CSVs, subtracting the amount that met this condition in 2015-2016 Direct Loan Program's CSV from the 2010-2011 Direct Loan Program's CSV.    

### _Question Three_

> In 2015, consider all the colonial colleges founded before the declaration of independence. Amongst these, what was the largest number of recipients within a school for either DL Graduate or DL Grad Plus loans?

Remember to provide your answer as a number. 

In [171]:
for i, c in enumerate(df15.columns): print(i, c)

0 OPE ID
1 School
2 State
3 Zip Code
4 School Type
5  DL SUBSIDIZED Recipients
6 # of Loans Originated
7 $ of Loans Originated
8 # of Disbursements
9 $ of Disbursements
10  DL UNSUBSIDIZED- UNDERGRADUATE Recipients
11 # of Loans Originated
12 $ of Loans Originated
13 # of Disbursements
14 $ of Disbursements
15  DL UNSUBSIDIZED- GRADUATE Recipients
16 # of Loans Originated
17 $ of Loans Originated
18 # of Disbursements
19 $ of Disbursements
20  DL PARENT PLUS Recipients
21 # of Loans Originated
22 $ of Loans Originated
23 # of Disbursements
24 $ of Disbursements
25 DL GRAD PLUSRecipients
26 # of Loans Originated
27 $ of Loans Originated
28 # of Disbursements
29 $ of Disbursements
30 Total $ Disbursements
31 Total $ of Loans Originated
32 Colonial College
33 FBS_Champs


In [148]:
colonials = ["Harvard University", "College of William", "Yale University",
             "Princeton University", "Columbia University", "University of Pennsylvania",
             "Brown University", "Rutgers", "Dartmouth College"]

df15["Colonial College"] = df15["School"].str.contains("|^".join(colonials), case=False)
df15[df15["Colonial College"]]

df15 = df15.replace("\$", "", regex=True)
df15 = df15.replace("^\s+", "", regex=True)
df15 = df15.replace("\s+$", "", regex=True)
df15 = df15.replace("-", "0", regex=True)

df15[" DL UNSUBSIDIZED- GRADUATE Recipients"] = df15[" DL UNSUBSIDIZED- GRADUATE Recipients"].replace(",", "", regex=True)
df15["DL GRAD PLUSRecipients"] = df15["DL GRAD PLUSRecipients"].replace(",", "", regex=True)

df15[" DL UNSUBSIDIZED- GRADUATE Recipients"] = df15[" DL UNSUBSIDIZED- GRADUATE Recipients"].astype(int)
df15["DL GRAD PLUSRecipients"] = df15["DL GRAD PLUSRecipients"].astype(int)
df15

max_grad_num = df15[df15["Colonial College"]][[" DL UNSUBSIDIZED- GRADUATE Recipients", 
                                               "DL GRAD PLUSRecipients"]].max(axis=1)
max_grad_num.max()

5488

### 3.   Answer = 5488 
 * **process**: First, it was important to research and determine which schools were considered Colonial Colleges. This was not linked to in the source documentation for this application set of questions, so I found a list from Wikipedia (reference: https://en.wikipedia.org/wiki/Colonial_colleges). To identify schools categorized as Colonial within the dataframe, I created a list of the schools designated as colonials, created a new column called "Colonial College" and stored whether the school was one of the colonials with a True or False by iterating through the "School" column and finding whether the string name of the school contained any of the matching string names within the "colonials" list by using an "or" (e.g. |) condition. Then, to find which Loan Type had more recipients among these schools, it required converting strings to numbers again. Since we were working in whole numbers (because people are measured in quantities of one) I converted the strings to ints as opposed to floats. This was performed on both the DL Graduate or DL Grad Plus loan types. Once both loan types were converted to numbers, I could then create a new variable to find and store the largest number of grad recipients (e.g. max_grad_num) among the two loan types by using the max method. This evaluated to find the largest number of recipients within a school for either DL Graduate or DL Grad Plus loans in the 2015-2016 Direct Loan Program CSV.   
 * **assumptions**: Earlier in the data cleaning process, we filtered the dataframe of schools whose final digit of the school's ZIP code is 3, 5, or 7. This eliminated a few of the colonial colleges from the list we were ultimately evaluating. It is possible that one of the other schools not included in the colonial colleges list had larger DL Graduate or DL Grad Plus loan recipients, but I am assuming that these were to be excluded since those schools' zip codes ended in 3, 5, or 7. Here, I am assuming that the desired evaluation is the largest number of recipients of either loan type among the remaining list of colonial colleges, and that the colonial college itself was not pertinent in the result. The list of colonial colleges also assumes the reference used to define which institutions fall under the colonial category (schools founded before 1776) is accurate, and that their updated/new name is contained in the CSV. 
 * **choices**: Again, it is extremely difficult to determine a series of choices for solving this question when you are not testing for efficiency of the output. As an amateur analyst and programmer, I have a difficult time finding reliable means to solve a problem in one way let alone multiple ways. I suppose you could have chosen an alternative way to store a new column containing a list of colonial colleges from the dataframe's "School" column; such as creating a python loop function to iterate through the dataframe's schools with an if, elif, and else condition that ultimately performed the same function of setting schools equal to True or False based on whether they were colonial colleges. This would be an alternative way to filter the list of schools to only the schools that contained the colonial college string names. Similarly, one could create other looping functions to strip special characters from the string in order to convert strings to ints. Lastly, you could create a loop function to iterate through the list of DL Graduate and DL Grad Plus loans to find the largest number of recipients among the list. I could have also used Series.str.match which would determine if each string matches a regular expression, but I opted for the less strict "contains" method so in case there was a slight difference in the school's naming conventions it wouldn't give it a false False designation.  
 * **reasoning**: I chose to use Pandas built in methods for the convenience and efficiency of writing interpretable code without the need for writing my own lengthy loop functions, which takes longer to write, is more likely to contain bugs, and increases the number of lines of code required to perform the same evaluation. 

### _Question Four_

> In 2015, consider all the private nonprofit schools in King county, WA. For these schools, consider the expected total loan amount if the loan is fully disbursed for unsubsidized graduate studies. Exclude all schools where the unsubsidized graduate loan amount is not available i.e. “-” or 0. What was the median value?
>
>**Note on median**: Median is defined as _the middle value among all the values being considered_. E.g. if the values being considered were 2, 9, 5, 7 and 15. Then, the median of these values will be 7. This is because there are exactly two values smaller than 7 - these are 2 and 5, and exactly two values greater than 7 - these are 9 and 15.

Remember to provide your answer as a number. 

In [144]:
king_zips = ["98001", "98001", "98001", "98002", "98003", "98003", "98004",
             "98004", "98004", "98004", "98004", "98005", "98006", "98007",
             "98008", "98009", "98010", "98011", "98013", "98013", "98014",
             "98015", "98019", "98022", "98023", "98023", "98024", "98025",
             "98027", "98028", "98028", "98029", "98030", "98031", "98032",
             "98033", "98034", "98035", "98038", "98039", "98040", "98041",
             "98042", "98042", "98045", "98047", "98047", "98050", "98051",
             "98052", "98053", "98054", "98055", "98056", "98056", "98057",
             "98058", "98059", "98059", "98062", "98063", "98063", "98064",
             "98065", "98068", "98068", "98070", "98071", "98072", "98073",
             "98074", "98074", "98075", "98075", "98083", "98092", "98093",
             "98093", "98101", "98102", "98103", "98104", "98105", "98106",
             "98107", "98108", "98108", "98109", "98111", "98112", "98114",
             "98115", "98116", "98117", "98118", "98119", "98121", "98122",
             "98124", "98125", "98126", "98131", "98132", "98133", "98133",
             "98134", "98136", "98138", "98138", "98144", "98145", "98146",
             "98146", "98148", "98148", "98148", "98148", "98148", "98154",
             "98155", "98155", "98155", "98155", "98155", "98158", "98158",
             "98160", "98161", "98164", "98166", "98166", "98166", "98168",
             "98168", "98168", "98168", "98171", "98174", "98177", "98177",
             "98178", "98178", "98188", "98188", "98188", "98198", "98198",
             "98198", "98198", "98199", "98224", "98288"]
df15[
    (df15["School Type"] == "Private0Nonprofit") &
    (df15["Zip Code"].str.contains("|".join(king_zips))) &
    (df15.iloc[:, 17] > 0)
].iloc[:, 17].median()

5889721.0

### 4.   Answer =  median value is 5,889,721

 * **process**: Like in question three, this required finding a reliable and comprehensive list of King County, WA Zip Codes, which was not provided in the application source materials. I relied on the following list (reference: http://bit.ly/2x7FoyO) set a new variable (king_zips) to store a list of King County, WA zip codes. Next I filtered the dataframe School Types to only look at Private Nonprofit institutions as the prompt calls for, combined with a conditional statement (e.g. &) to also filter schools only contained in the king_zips variable, and to only include the \$ of Loans Originated where it was greater than 0. Lastly, I used the median method on the remaining list of Private Nonprofit schools that met all three conditions. This eliminated the need to sort the values in order before selecting for the median value. 
 * **assumptions**: I am assuming that the variable named "king_zips" is the exhausted list of King County Zip Codes and I am not missing any that would otherwise produce a different result. The result also assumes that it is absent of zip codes with the final digits 3, 5, 7. 
 * **choices**: There was a way to use OPE ID and Zip Codes to filter the dataframe to a list of likely Zip Codes that only pertained to King County, but comparing the list side by side, defining a list of King County Zip Codes was more comprehensive by one school in the list. Another choice I opted for when writing this evaluation, rather than write out the column name referencing the unsubsidized graduate of fully disbursed loan amounts, I used the Pandas iloc (index location) method because I am trying to select a column by an integer. I considered using "DataFrame.ix", which is a label-location based indexer, with integer position fallback, but because in this case the axis is integer based, Pandas documentation stated ONLY label based access and not positional access is supported.  
 * **reasoning**: In solving this problem, it’s usually better to be explicit and use .iloc, which is what I did here. Using the .iloc method over a column range, I could reference this column's location within the dataframe to make it easier to write, but this did not alter the basic problem solving methodology. It just made the code easier to write and more efficient to read. 

### _Question Five_

More data cleaning is involved! 
> Consider all the schools who were football champions in the NCAA from 2001 to 2015 (both years included) in the FBS division. For these schools (only include main campuses that participate in football and not all campuses) consider the expected total loan amount if the loan is fully disbursed for GRAD PLUS loans in 2015 data provided. Calculate the sum of these amounts.

Remember to provide your answer as a number. 

In [151]:
for i, c in enumerate(df15.columns): print(i, c)

0 OPE ID
1 School
2 State
3 Zip Code
4 School Type
5  DL SUBSIDIZED Recipients
6 # of Loans Originated
7 $ of Loans Originated
8 # of Disbursements
9 $ of Disbursements
10  DL UNSUBSIDIZED- UNDERGRADUATE Recipients
11 # of Loans Originated
12 $ of Loans Originated
13 # of Disbursements
14 $ of Disbursements
15  DL UNSUBSIDIZED- GRADUATE Recipients
16 # of Loans Originated
17 $ of Loans Originated
18 # of Disbursements
19 $ of Disbursements
20  DL PARENT PLUS Recipients
21 # of Loans Originated
22 $ of Loans Originated
23 # of Disbursements
24 $ of Disbursements
25 DL GRAD PLUSRecipients
26 # of Loans Originated
27 $ of Loans Originated
28 # of Disbursements
29 $ of Disbursements
30 Total $ Disbursements
31 Total $ of Loans Originated
32 Colonial College


In [166]:
# non-repeating list of FSB NCAA Champions from 2001-2015 (inclusive) using this wikipedia list: 
# http://bit.ly/1Katpwl, but referencing school names, not teams. 

FBS_NCAA_Champs = ["University of Miami", "Ohio State University", "University of Southern California", 
                   "Louisiana State University", "University of Oklahoma", "University of Texas at Austin",
                  "University of Florida", "University of Missouri", "University of Utah", 
                   "University of Alabama", "Auburn University", "Texas Christian University", 
                   "Oklahoma State University", "University of Notre Dame", "Florida State University"]

FBS_NCAA_Champs = [x.upper() for x in FBS_NCAA_Champs] 

df15 = df15.replace("\$", "", regex=True)
df15 = df15.replace("^\s+", "", regex=True)
df15 = df15.replace("\s+$", "", regex=True)
df15 = df15.replace("-", "0", regex=True) 

df15["$ of Loans Originated"] = df15["$ of Loans Originated"].replace(",", "", regex=True)
df15["$ of Loans Originated"] = df15["$ of Loans Originated"].astype(float)
df15[df15["School"].isin(FBS_NCAA_Champs)].iloc[:, 27].sum()

194250679.0

### 5.   Answer =  194,250,679 dollars

 * **process**: To solve this problem, I found a list of NCAA FBS Champions using this reference (http://bit.ly/1Katpwl) and selected schools from years 2001 through 2015. I made sure to only select for unique school names as there were some schools that won multiple times over this 15 year period. Next, I created a variable (FBS_NCAA_Champs) to store this list of unique school names. In order to find these schools among the Direct Loan Program 2015-2016 CSV, I performed a method for capitolizing the letters of each character within each string through the entire variable list so that the CSV school names would exactly match the list I was comparing it to. Just as in previous questions, I had to convert the data within the "\$ of Loans Originated" column from strings to floats in order to perform the sum method, so I removed special characters from each cell row. Then I was able to iterate through the column of school names to find whether the FBS_NCAA_Champs string was in the school string giving me only the FBS NCAA Champions. My last step was to sum only \$ of Loans Originated for the DL Grad Plus Recipients, otherwise, I would have summed all \$ of Loans Originated columns across all Loan Types). I used the .iloc method to filter data from the specific index within the dataframe (all rows, within the column with index location 27, which was the location of the DL GRAD PLUSRecipients column, and sum only this column, providing my answer.
 * **assumptions**: Again, several FBS NCAA Champion Schools were not included in this sum because their zip codes ended in 3,5,7 which excluded them from the analysis. 
 * **choices**: without given source material to verify the list of FBS NCAA Champions, I had to find an outside source to compile a list. I chose to use a regular expression to find and replace special characters in order to cast the string to a float, allowing me to perform a sum down the column of $ of Loans Originated at its specific location in the dataframe.   
 * **reasoning**: Using an index location methodology proved effective in other instances, which is why I chose to repeat using it in this problem. I used a regular expression to cast the string to a float, which enabled me to perform a sum. And lastly, without source material provided in the prompt, it was necessary for me to create my own variable that contained a list of all schools who won the FBS NCAA Championships in order to filter the data to only these scools.  